In [ ]:
import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.insert(0, os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.append(os.path.join(os.path.join(os.getenv('HOME'), 'ASL') + '/src'))

from utils_asl import label_to_png, png_to_label
from pathlib import Path
import numpy as np


scene = "scene0003_00"

In [ ]:
p = f"/home/jonfrey/Datasets/labels_generated/new_format_scene0-10/scans/{scene}"
paths = [str(s) for s in Path(p).rglob('*.png') if  str(s).find('identifier')]

la = lambda p: int(p.split('/')[-1][:-4])
paths.sort( key = la)


In [ ]:
scannet = "/home/jonfrey/Datasets/scannet/scans"

In [ ]:
from visu import Visualizer   
from PIL import Image
from utils_asl import load_label_scannet, load_mapping_scannet

mapping_scannet = load_mapping_scannet("/home/jonfrey/Datasets/scannet/scannetv2-labels.combined.tsv")


visu = Visualizer('/home/jonfrey/tmp', num_classes=40)

def get_image(visu, pa, scannet, confidence=0.9, factor_of_mean = None, return_gt=False,mapping_scannet=False):
    image_pth = '/'.join([scannet, pa.split('/')[-3],  'color', pa.split('/')[-1].replace('png','jpg')])
    label_soft = png_to_label(pa)
    label = np.argmax( label_soft, axis = 2 ) + 1
    
    val = np.max(label_soft, axis=2)
    
    if factor_of_mean is not None:
        confidence = val.mean() * factor_of_mean 
    
    label[ val < confidence ] = 0
    img = np.array( Image.open( image_pth ) )      
    ret =  (visu.plot_detectron( img, label, store=False, jupyter=False, alpha=0.4) , img)
    if return_gt:
        label_pth = '/'.join([scannet, pa.split('/')[-3],  'label-filt', pa.split('/')[-1]])
        gt = load_label_scannet(label_pth, mapping_scannet)
        ret += (visu.plot_detectron( img, gt, store=False, jupyter=True, alpha=0.4), )
    return ret


In [ ]:
from skimage.data import astronaut
from skimage.segmentation import slic
from skimage.util import img_as_float
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries

for j,pa in enumerate( paths):
    img, img_ori, gt = get_image(visu, pa, scannet, confidence=0.1, factor_of_mean=None, return_gt=True,mapping_scannet =mapping_scannet )
    
    segments_slic = slic(img_as_float(img_ori), n_segments=250, compactness=20, sigma=5,
                         start_label=1)
    fig, ax = plt.subplots(1, 1, figsize=(10, 10), sharex=True, sharey=True)
    
    ax.imshow(mark_boundaries(img_as_float(img), segments_slic))
    plt.tight_layout()
    plt.show()
    
    if j >= 0:
        break
#   r, g, b = img.split()
#   img = np.array( Image.merge("RGB", (b, g, r)) )
#   video.write(img)
#   print(j)
# video.release()


In [ ]:
from cv2 import VideoWriter, VideoWriter_fourcc
import cv2
FPS = 10
height,width,_ = np.array( cv2.imread(paths[0] )).shape
fourcc = VideoWriter_fourcc("M", "J", "P","G")
co = 0.5
video = VideoWriter(f'/home/jonfrey/Documents/master_thesis/weekly/21_05_31/{scene}_conf_{co}_pred.avi', fourcc, float(FPS), (width, height))

for j,pa in enumerate( paths):
    img, img_ori, gt = get_image(visu, pa, scannet, confidence=co, factor_of_mean=None, return_gt=True,mapping_scannet =mapping_scannet )

    img = Image.fromarray(img)
    
    r, g, b = img.split()
    img = np.array( Image.merge("RGB", (b, g, r)) )
    video.write(img)
    print(j)
        
video.release()